# PTSD Mouse Data 

The total data consist of two data sets, measuring responses from 20 different mice at 4 timepoints each: Baseline, Pre-fear exposure, immediately after the Fear exposure, and 9 days later (D9). 10 of these mice are seratonin transporter knockouts (KO) and 10 are wildtype (WT). 

The data are: 
- 79 MRIs. The pre-fear, fear, and D9 images are MN(II) enhanced MRIs (MEMRI). The MEMRI images are used to measure neuronal functioning - (when neurons are active, their uptake of MN(II) is increased?). The baseline images, which are regular fMRI, are 
- 79 measurements of percent time spent in the light by the mice. 

In both data sets, the KO_04_D9 datapoint is missing (hence 79 instead of 80). 

In [3]:
# Import needed packages for analysis
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

import cfl_examples.lesion_mapping.brain_util as BU
import cfl_examples.lesion_mapping.brain_vis as BV

# load response data 
Y = pd.read_pickle('Y.pkl')

# MRI Data 
Some facts about the data: 
- the dimensions of the brain box in each image are (124, 200, 82) (2,033,600 voxels total)
- the images are originally in RPS orientation. We flip them to RAS orientation because then they have the same alignment as some other MRIs we've been looking at 

In [4]:
behav_csv = 'PTSD_Data_Share\Behavior_data\PTSD_PerLight.csv'
mri_dir = 'PTSD_Data_Share\MEMRI_data'

# load one image to check out its dimensions
img = BU.load_brain(os.path.join(mri_dir, "PTSD_KO_03_BL.nii"))
mri_dims = img.shape

In [5]:
# load all the images in RAS orientation 
X, Y_unused = BU.load_data(mri_dir, behav_csv, mri_dims, ori='RAS')

# Load Masks
We were given two masks to fit the the MRIs, a linearly aligned- and non-linearly aligned mask. These masks tell which voxels in the image are part of the brain vs which are empty space. 

The difference between the non-linear and linear mask (from an email from Taylor): "The non-linearly aligned mask is better aligned to the data, however the non-linear "warping" creates artifacts near the surface of the brain that may result in grabbing more undesired non-brain voxels when masking. The linearly aligned mask may not align to the surface of the brain as well and may miss or cut out brain tissue, but may not grab as many non-brain voxels. 

The non-linear mask leaves 531,632 voxels and the linear mask 482,793 voxels unmasked. 

In [6]:
# load the non-linear mask template
nl_mask_path = os.path.join('PTSD_Data_Share/templates\MuseTemplate_nonlinear_mask.nii')
nl_mask = BU.load_brain(nl_mask_path)
nolin_mask_vec = BU.flatten(nl_mask)

# load the linear mask template
l_mask_path = os.path.join('PTSD_Data_Share/templates\MuseTemplate_linear_mask.nii')
l_mask = BU.load_brain(l_mask_path)
lin_mask_vec = BU.flatten(l_mask)

# Heatmaps, WT vs KO 

These heatmaps show the average value of each voxel for the timepoint, by each genotype  

In [8]:
baseline_indices = Y[Y["Timepoint"]=="BL"].index.tolist()
prefear_indices = Y[Y["Timepoint"]=="PreF"].index.tolist()
postfear_indices = Y[Y["Timepoint"]=="Fear"].index.tolist()
d9_indices = Y[Y["Timepoint"]=="D9"].index.tolist() 
timepoints_dir = {'BL' : baseline_indices, 'PreF': prefear_indices, "Fear": postfear_indices, 'D9': d9_indices}


WT_indices = Y[Y["Genotype"]=="WT"].index.tolist()
KO_indices = Y[Y["Genotype"]=="KO"].index.tolist() 


In [14]:
# from tqdm import tqdm

# def empty_timepoint_dir(): 
#     return {'BL' : np.zeros(np.prod(mri_dims)), 'PreF': np.zeros(np.prod(mri_dims)), "Fear": np.zeros(np.prod(mri_dims)), 'D9': np.zeros(np.prod(mri_dims))}


# # create wild type heatmap for each timepoint 
# WT_HM_dir = empty_timepoint_dir()
# for timepoint in timepoints_dir: 
#     # create empty array for heatmap 
#     currentHM = np.zeros(np.prod(mri_dims))

#     #get all the relevant MRI images 
#     indices = list(set(WT_indices).intersection(set(timepoints_dir[timepoint])))
#     n = len(indices)
#     #add to the heatmap 
#     for brain in tqdm(X[indices]):
#         currentHM += brain

#     # divide by number of images to get average 
#     np.divide(currentHM, n)
#     WT_HM_dir[timepoint] = currentHM

#     save_name = 'wt_heatmap_' + timepoint + '.npy'
#     np.save(os.path.join(save_name), currentHM) 


# # create KO heatmap for each timepoint
# KO_HM_dir = empty_timepoint_dir()
# for timepoint in timepoints_dir: 
#     # create empty array for heatmap 
#     currentHM = np.zeros(np.prod(mri_dims))

#     #get all the relevant MRI images 
#     indices = list(set(KO_indices).intersection(set(timepoints_dir[timepoint])))
#     n = len(indices)
#     #add to the heatmap 
#     for brain in tqdm(X[indices]):
#         currentHM += brain

#     # divide by number of images to get average 
#     np.divide(currentHM, n)
#     KO_HM_dir[timepoint] = currentHM

#     save_name = 'ko_heatmap_' + timepoint + '.npy'
#     np.save(os.path.join(save_name), currentHM) 


100%|██████████| 9/9 [00:00<00:00, 249.75it/s]


In [28]:
# load WT heatmaps 
heatmap_names = ['wt_heatmap_' + str(timepoint) for timepoint in timepoints_dir.keys()]
HM_list = []
for heatmap in heatmap_names: 
    HM_list.append(np.load(heatmap + '.npy'))


# specify labels for plot (note the labels below are specifically for RAS orientation)
dir_labels = { 'saggital' :   ['P', 'A', 'D', 'V'],
               'coronal' :    ['L', 'R', 'D', 'V'],
               'horizontal' : ['L', 'R', 'A', 'P']} 

# display WT heatmaps 
BV.plot_interactive_panels(np.vstack(HM_list), mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=["WT " + str(timepoint) for timepoint in timepoints_dir])

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

In [27]:
# load KO heatmaps 
heatmap_names = ['ko_heatmap_' + str(timepoint) for timepoint in timepoints_dir.keys()]
HM_list = []
for heatmap in heatmap_names: 
    HM_list.append(np.load('ko_heatmap_' + '.npy'))


# specify labels for plot (note the labels below are specifically for RAS orientation)
dir_labels = { 'saggital' :   ['P', 'A', 'D', 'V'],
               'coronal' :    ['L', 'R', 'D', 'V'],
               'horizontal' : ['L', 'R', 'A', 'P']} 

#display KO heatmaps 
BV.plot_interactive_panels(np.vstack(HM_list), mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=["KO " + str(timepoint) for timepoint in timepoints_dir])

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

In [32]:
# load all heatmaps to display side-by-side
all_HMs = []
for timepoint in timepoints_dir: 
    all_HMs.append(np.load('wt_heatmap_' + timepoint + '.npy'))
    all_HMs.append(np.load('ko_heatmap_' + timepoint + '.npy'))


# specify labels for plot (note the labels below are specifically for RAS orientation)
dir_labels = { 'saggital' :   ['P', 'A', 'D', 'V'],
               'coronal' :    ['L', 'R', 'D', 'V'],
               'horizontal' : ['L', 'R', 'A', 'P']} 

#display BL and pre-fear heatmaps 
BV.plot_interactive_panels(np.vstack(all_HMs[:4]), mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=["WT BL", "KO BL", "WT Pre-fear", "KO Pre-fear"])

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

In [30]:
#display BL and pre-fear heatmaps 
BV.plot_interactive_panels(np.vstack(HM_list[4:]), mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=["WT Fear", "KO Fear", "WT D9", "KO D9"])

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

## Differences Between Wild-type and KO heatmaps 

In [33]:
# WT - KO heatmaps 
all_HMs = [HM_list[1] - HM_list[0], HM_list[3] - HM_list[2], HM_list[5] - HM_list[4], HM_list[7] - HM_list[6]]

#display BL and pre-fear heatmaps 
BV.plot_interactive_panels(np.vstack(all_HMs), mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=["BL diffs", "Pre-fear Diffs", "Fear Diffs", "D9 Diffs"])

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

# Baseline Adjustment

In this section, images with the baseline MRI subtracted off are shown. 

In [113]:
# function to subtract off the baseline 
def adjust_image(X, id, timepoint): 
    '''input: X (array of MRI images), ID of mouse and timepoint to use ("PreF", "Fear", "D9")

    returns baseline-adjusted image
    '''
    # get image for that mouse at that timepoint 
    image_index = Y.loc[(Y.ID==id) & (Y.Timepoint==timepoint)].index[0]
    image = X[image_index]

    # get image for that mouse at baseline 
    baseline_index = Y.loc[(Y.ID==id) & (Y.Timepoint=="BL")].index[0]
    baseline_image = X[baseline_index]

    # subtract baseline from timepoint image 
    adjusted_mri = image - baseline_image

    return adjusted_mrir4

## Heatmap for each timepoint, adjusted by baseline intensity

In [116]:
HM_dir = {'PreF': np.zeros(np.prod(mri_dims)), "Fear": np.zeros(np.prod(mri_dims)), 'D9': np.zeros(np.prod(mri_dims))}
all_mouse_indices = pd.unique(Y["ID"])


for timepoint in HM_dir:
    # for each mouse....
    for mouse_id in all_mouse_indices:
        # get the image associated with that timepoint, subtract off the baseline
        if mouse_id != 1 or timepoint != 'D9': #don't do this for the missing value
            adjusted_mri = adjust_image(X, mouse_id, timepoint)
            # add the result to the correct heatmap entry 
            HM_dir[timepoint] += BU.flatten(adjusted_mri)
    # divide to get the average 
    n = len(timepoints_dir[timepoint])
    np.divide(HM_dir[timepoint], n) 

In [117]:
BV.plot_interactive_panels(np.vstack(list(HM_dir.values())), mri_dims, nolin_mask_vec, figsize=(12, 3), dir_labels=dir_labels, column_titles=list(HM_dir.keys()))


interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

# Variance
This plot shows the level of variance between individual voxels in all the MRIs for each timepoint.

In [112]:
var_dir = timepoints_dir.copy()
for timepoint in timepoints_dir: 

    current_var = np.var(X[timepoints_dir[timepoint]], axis=0)
    var_dir[timepoint] = current_var

BV.plot_interactive_panels(np.vstack(var_dir.values()), mri_dims, nolin_mask_vec, figsize=(15, 5), dir_labels=dir_labels, colormap="Reds", step= 1, column_titles=["Baseline Variance", "Pre-fear Variance", "Fear Variance", "D9 Variance"]) 

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=123), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=199), Output(…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=81), Output()…

 The variance tends to be highest on the edges of the brain or in small regions inside of the brain. I believe these regions correspond with the regions of high activity seen above in the individual MRIs. The variance seems to be fairly constant across mice. 

# Coarsening

In the following code, we coarsen the mouse MRIs in order to reduce their dimensionality to more manegeable sizes. The coarsening is done by collapsing n x n cubes of voxels into a single voxel, where n is referred to as the 'coarsening factor'. The new single voxel contains the average value of the previous cube of voxels. 

In [135]:
def new_coarsen(x, coarsening_factor, brain_dims):
    ''' coarsens brain by specified amount
    arguments:
        x: 3D array of brain or 1D array of flattened brain
        coarsening_factor: how many original voxels to include in new voxel along a dimension
        brain_dims: array containing [dim0, dim1, dim2] of 3D brain (int array)
    returns:
        new_x: 3D array of coarsened brain
    '''
    # make 3D if flattened
    if x.ndim < 3:
        x = BU.unflatten(x, brain_dims)

    brain_dims = np.array(brain_dims)
    new_dims = np.ceil(brain_dims/coarsening_factor).astype(int)
    new_x = np.zeros(new_dims)
    for i in range(new_dims[0]):
        for j in range(new_dims[1]):
            for k in range(new_dims[2]):
                # select all voxels to contribute to coarsened voxel (i,j,k)
                istart = i*coarsening_factor
                jstart = j*coarsening_factor
                kstart = k*coarsening_factor
                iend = np.min((istart+coarsening_factor, brain_dims[0])) # handle edge case
                jend = np.min((jstart+coarsening_factor, brain_dims[1]))
                kend = np.min((kstart+coarsening_factor, brain_dims[2]))

                sample = x[istart:iend, jstart:jend, kstart:kend]


                # average 
                new_x[i, j, k] = np.average(sample)
    return new_x


 ## Correlation between each voxel of brains and Y (percent time spent in the light)


For each voxel (after coarsening): 
- have 20 pairs of values: MRI (fear - pre-fear)  and Y (fear- pre-fear)
- want to find correlation for ^ for each voxel 
- graph back onto the brain 

In [169]:
# n_images = 79

# # new array of coarsened brain images 
# X_new = np.zeros((n_images, np.prod(coarsened_brain.shape)))
# for i in range(n_images): 
#     X_new[i] = BU.flatten(new_coarsen(X[i], coarsening_factor, np.array(mri_dims)))

# np.save(os.path.join('coarsened3_X.npy'), X_new)

In [35]:
from scipy.stats import pearsonr
X_new = np.load(os.path.join('coarsened3_X.npy'))

Y_diffs = Y.PerLight[timepoints_dir["Fear"]].to_numpy() - Y.PerLight[timepoints_dir["PreF"]].to_numpy()

X_diffs = X_new[timepoints_dir["Fear"]] - X_new[timepoints_dir["PreF"]]

X_diffs.shape
# corrs = np.zeros( X_new.shape[1], dtype=np.int16)


# # create correlations for WT only

# for voxel in range(X_new.shape[1]): 
#     value = pearsonr(Y_diffs, X_diffs[:, voxel])[0]
#     if np.isnan(value): 
#         value = 0 
#     corrs[voxel] = value

# #note: when everything is a 0, pearsonr returns a nan correlation value

(20, 78792)

In [180]:
np.save(os.path.join("correlations.npy"), corrs)

In [133]:
BV.plot_interactive_panels(corrs, coarsened_brain.shape, coarsened_nl_mask, figsize=(12, 5), dir_labels=dir_labels, std_scale='corr', column_titles=["Correlations for Coarsened Brain"], step=1)

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=24), Output()…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=39), Output()…

interactive(children=(IntSlider(value=0, continuous_update=False, description='brain_slice', max=16), Output()…

My take on the above graph: 
It's hard to tell if there's anything there. Note that the scale only varies between +- 0.6 (no extremely strong correlations )

In [ ]:
########## Extra stuff 



## Mean and Variance side-by-side

# display heatmaps 
# BV.plot_interactive_panels(np.vstack((HM_list[0], var_dir['BL'], HM_list[1], var_dir['PreF'])), mri_dims, nolin_mask_vec, figsize=(17, 3), dir_labels=dir_labels, column_titles=["BL Mean", "BL variance", "Pre-fear Mean", "Pre-fear Var"])





## Heatmap of the WT vs KO mice


# WT_indices = Y[Y["Genotype"]=="WT"].index.tolist()
# KO_indices = Y.dropna(how='any', axis=0)[Y["Genotype"]=="KO"].index.tolist() #remove last value bc it doesn't have an associated MRI 

# # create wild type heatmap 
# heatmap_WT = np.zeros(np.prod(mri_dims))
# n = X[WT_indices].shape[0]
# for brain in tqdm(X[WT_indices]):
#     heatmap_WT += brain
# np.divide(heatmap_WT, n)
# np.save(os.path.join('heatmap_WT.npy'), heatmap)


# # create KO heatmap 
# heatmap_KO = np.zeros(np.prod(mri_dims))
# n = X[KO_indices].shape[0]
# for brain in tqdm(X[KO_indices]):
#     heatmap_KO += brain
# np.divide(heatmap_KO, n) 
# np.save(os.path.join('heatmap_KO.npy'), heatmap)


# heatmap_WT = np.load(os.path.join('heatmap_WT.npy'))
# heatmap_KO = np.load(os.path.join('heatmap_KO.npy'))

# BV.plot_interactive_panels(np.vstack((heatmap_WT, heatmap_KO)), mri_dims, nolin_mask_vec, figsize=(10, 5), dir_labels=dir_labels, column_titles=["Wild-type", "Knockout"], step=1)